In [ ]:
from google.colab import drive
drive.flush_and_unmount()
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import time
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf 
import tensorflow.keras as keras
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Main Code provided by the paper

## Data loading and pre-processing

### Data Loading

In [ ]:
!7z x '/content/gdrive/MyDrive/Colab Notebooks/MiniProject4/dataset.7z'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/gdrive/MyDrive/Colab Notebooks/MiniProject4/                                                               1 file, 437541578 bytes (418 MiB)

Extracting archive: /content/gdrive/MyDrive/Colab Notebooks/MiniProject4/dataset.7z
--
Path = /content/gdrive/MyDrive/Colab Notebooks/MiniProject4/dataset.7z
Type = 7z
Physical Size = 437541578
Headers Size = 459592
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      0% 84         0% 557 - dataset/modelnet2d/airplane/1/525.png                                                

### Pre-processing

In [ ]:
import numpy as np
import os
from PIL import Image


def getPath(which_set, filetype, dirname='data'):
    """
    Getting the path for the desired dataset.
    which_set: train, test
    filetype: dat, cat, info
    """     
    if which_set == 'train':
        instance_list = '46789'
    elif which_set == 'test':
        instance_list = '01235'
    filename = 'smallnorb-5x%sx9x18x6x2x96x96-%s-%s.mat' % \
        (instance_list, which_set + 'ing', filetype)
    return os.path.join(dirname, filename)


def readNums(file_handle, num_type, count):
    """
    Reads 4 bytes from file, returns it as a 32-bit integer.
    """
    num_bytes = count * np.dtype(num_type).itemsize
    string = file_handle.read(num_bytes)
    return np.fromstring(string, dtype=num_type)


def readHeader(file_handle, debug=False):
    """
    Reads the header of the file.
    file_handle: an open file handle.
    returns: data type, element size, rank, shape, size
    """

    key_to_type = {0x1E3D4C51: ('float32', 4),
                   # 0x1E3D4C52 : ('packed matrix', 0),
                   0x1E3D4C53: ('float64', 8),
                   0x1E3D4C54: ('int32', 4),
                   0x1E3D4C55: ('uint8', 1),
                   0x1E3D4C56: ('int16', 2)}

    type_key = readNums(file_handle, 'int32', 1)[0]
    elem_type, elem_size = key_to_type[type_key]
    if debug:
        print("header's type key, type, type size: ",
              type_key, elem_type, elem_size)
    if elem_type == 'packed matrix':
        raise NotImplementedError('packed matrix not supported')

    num_dims = readNums(file_handle, 'int32', 1)[0]
    if debug:
        print('# of dimensions, according to header: ', num_dims)

    shape = np.fromfile(file_handle,
                        dtype='int32',
                        count=max(num_dims, 3))[:num_dims]

    if debug:
        print('Tensor shape, as listed in header:', shape)

    return elem_type, elem_size, shape


def parseNORBFile(file_handle, debug=False):
    """
    Parse file into numpy array and return.
    file_handle: an open file handle.
    """
    elem_type, elem_size, shape = readHeader(file_handle, debug)
    beginning = file_handle.tell()
    num_elems = np.prod(shape)
    result = np.fromfile(file_handle,
                         dtype=elem_type,
                         count=num_elems).reshape(shape)
    return result


def get_data(which_set='train', file_type='dat', path='data', debug=True):
    file_path = getPath(which_set, file_type, path)
    file_handle = open(file_path, 'rb')
    norb_data = parseNORBFile(file_handle, debug)
    return norb_data

In [ ]:
dataset_path = "dataset/smallnorb/"
train_info = get_data('train', 'info', dataset_path, debug=False)
raw_train_data = get_data('train', 'dat', dataset_path, debug=False)
raw_train_label = get_data('train', 'cat', dataset_path, debug=False)

test_info = get_data('test', 'info', dataset_path, debug=False)
raw_test_data = get_data('test', 'dat', dataset_path, debug=False)
raw_test_label = get_data('test', 'cat', dataset_path, debug=False)

train_info = np.concatenate([raw_train_label[:, np.newaxis], train_info], axis=-1)
test_info = np.concatenate([raw_test_label[:, np.newaxis], test_info], axis=-1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [ ]:
data = np.concatenate([raw_train_data, raw_test_data], axis=0)
label = np.concatenate([raw_train_label, raw_test_label], axis=0)
info = np.concatenate([train_info, test_info], axis=0)

In [ ]:
def sort_info(data, label, info):
    feat_order = info.view('i4,i4,i4,i4,i4').argsort(kind='mergesort', order=['f0', 'f3', 'f2', 'f1', 'f4'], axis=0).reshape(-1)
    data, label = data[feat_order], label[feat_order]
    info = info[feat_order]#[label_order]
    return data, label, info

In [ ]:
all_data, all_label, all_info = sort_info(data, label, info)

In [ ]:
def split_train_test(data):
    num_classes = 5
    data_per_class = len(data) // num_classes
    viewpoint = 18
    train_viewpoint = 6 / 18
    
    train_idx, test_idx = [], []
    for i in range(num_classes):
        train_idx.extend(range(i * data_per_class, i * data_per_class + int(data_per_class * train_viewpoint)))
        test_idx.extend(range(i * data_per_class + int(data_per_class * train_viewpoint), (i+1) * data_per_class))
    return train_idx, test_idx

train_idx, test_idx = split_train_test(all_data)
train_data, test_data = all_data[train_idx], all_data[test_idx]
train_label, test_label = all_label[train_idx], all_label[test_idx]
train_info, test_info = all_info[train_idx], all_info[test_idx]

### Preparing binocular images

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = 48
def get_binocular_dataset(data, label, batch_size=BATCH_SIZE):
    def preprocess_image(left_image, right_image):
        left_image = tf.expand_dims(left_image, -1)
        left_image = tf.image.resize(left_image, [IMG_SIZE, IMG_SIZE])
        left_image /= 255.0

        right_image = tf.expand_dims(right_image, -1)
        right_image = tf.image.resize(right_image, [IMG_SIZE, IMG_SIZE])
        right_image /= 255.0  # normalize to [0,1] range
        return left_image, right_image

    def load_and_preprocess_image(binocular_images):
        left_image, right_image = binocular_images[0], binocular_images[1]
        return preprocess_image(left_image, right_image)

    # The tuples are unpacked into the positional arguments of the mapped function 
    def load_and_preprocess_from_path_label(data_path, label):
        return load_and_preprocess_image(data_path), label
    
    ds = tf.data.Dataset.from_tensor_slices((data, label))
    ds = ds.map(load_and_preprocess_from_path_label, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.shuffle(buffer_size=len(data))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    print(ds)
    return ds

In [ ]:
train_ds = get_binocular_dataset(train_data, train_label)
test_ds = get_binocular_dataset(test_data, test_label)

<PrefetchDataset shapes: (((None, 48, 48, 1), (None, 48, 48, 1)), (None,)), types: ((tf.float32, tf.float32), tf.int32)>
<PrefetchDataset shapes: (((None, 48, 48, 1), (None, 48, 48, 1)), (None,)), types: ((tf.float32, tf.float32), tf.int32)>


## CNN2

### Defining Monocular layer including cmpooling and Convolution

In [ ]:
## Utility function
def cmpooling(fmaps, scale_list, pool_stride):
    # make sure the scale_list is in decending order
    if scale_list[0] - scale_list[1] < 0:
        scale_list = scale_list[::-1]
        
    # concentric multi-scale pooling
    offset = [0] + [-(scale_list[i+1] - scale_list[0])//2 for i in range(len(scale_list) - 1)]
    pool_maps = []
    for offset, scale in zip(offset, scale_list):
        slice_maps = tf.slice(fmaps, [0, offset, offset, 0], [-1, fmaps.shape[1]-offset*2, fmaps.shape[2]-offset*2, -1])
        pool_map = tf.nn.max_pool2d(slice_maps, scale, pool_stride, "VALID")
        pool_maps.append(pool_map)
    
    # assert same shape for all pool_map
    for i in range(len(pool_maps)-1):
        assert pool_maps[i].shape[1:] == pool_maps[-1].shape[1:]
    return pool_maps

# Concat the feature maps in different scale and convolution once. (paper version)
class Monocular(tf.keras.layers.Layer):
    def __init__(self, filters, ksize, **kwargs):
        super(Monocular, self).__init__(**kwargs)
        self.filters = filters
        self.ksize = ksize
#         self.conv = tf.keras.layers.Conv2D(filters, ksize, input_shape=kwargs['input_shape'], activation='relu', padding='same')
    
    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(self.filters, self.ksize, input_shape=input_shape, activation='relu', padding='same')
    
    def call(self, fmaps, scale_list, pool_stride):
        pool_maps = cmpooling(fmaps, scale_list, pool_stride)
        pool_maps = tf.concat(pool_maps, axis=-1)
        return self.conv(pool_maps)

### Defining CNN2 Model

In [ ]:
SCALE_LIST = [1,3,5]
def CNN2(input_shape, num_classes, scale_list):
    left_eye = tf.keras.Input(input_shape, name='left_eye')
    right_eye = tf.keras.Input(input_shape, name='right_eye')
    
    # parallax augmentation
    parallax = left_eye - right_eye 
    left = tf.concat([left_eye, -parallax], axis=-1)
    right = tf.concat([right_eye, parallax], axis=-1)
    # 
    left1 = Monocular(6, 5, input_shape=input_shape, name='mono1_left')(left, scale_list=scale_list, pool_stride=2)
    right1 = Monocular(6, 5, input_shape=input_shape, name='mono1_right')(right, scale_list=scale_list, pool_stride=2)
    
    left2 = Monocular(12, 5, name='mono2_left')(tf.concat([left1, right1], axis=-1), scale_list=scale_list, pool_stride=1)
    right2 = Monocular(12, 5, name='mono2_right')(tf.concat([right1, left1], axis=-1), scale_list=scale_list, pool_stride=1)
    
    left3 = Monocular(32, 3, name='mono3_left')(tf.concat([left2, right2], axis=-1), scale_list=scale_list, pool_stride=1)
    right3 = Monocular(32, 3, name='mono3_right')(tf.concat([right2, left2], axis=-1), scale_list=scale_list, pool_stride=1)
    
    x = tf.concat([left3, right3], axis=-1)
    x = tf.keras.layers.Conv2D(256, 3, strides=1, activation='relu', name='conv1')(x)
    x = tf.keras.layers.Conv2D(256, 1, strides=1, activation='relu', name='conv2')(x)
    x = tf.keras.layers.Conv2D(64, 1, strides=1, activation='relu', name='conv3')(x)
    feature_vector = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.7)(x)
    predicted_output = tf.keras.layers.Dense(num_classes, activation='softmax', name='output')(feature_vector)
    
    return tf.keras.Model([left_eye, right_eye], predicted_output)

In [ ]:
def create_model(model, input_shape, num_classes, scale_list):
    m = model(input_shape, num_classes, scale_list)
    # learning rate schedule
    initial_learning_rate = 0.0001
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=100000,
                                                              decay_rate=0.96, staircase=True)
    
    # compile the model
    m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),  # Optimizer
                  # Loss function to minimize
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  # List of metrics to monitor
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
                 )
    return m

### Create and fitting the model

In [ ]:
start = time.time()
cnn2 = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, 1), num_classes=5, scale_list=SCALE_LIST)
cnn2.fit(train_ds , epochs=10, validation_data=test_ds)
end = time.time()
print(f'Running time: {end-start}')

Epoch 1/10
507/507 [==============================] - 44s 22ms/step - loss: 0.9980 - sparse_categorical_accuracy: 0.5822 - val_loss: 0.5336 - val_sparse_categorical_accuracy: 0.7996
Epoch 2/10
507/507 [==============================] - 12s 21ms/step - loss: 0.2212 - sparse_categorical_accuracy: 0.9177 - val_loss: 0.5423 - val_sparse_categorical_accuracy: 0.8229
Epoch 3/10
507/507 [==============================] - 11s 20ms/step - loss: 0.1248 - sparse_categorical_accuracy: 0.9535 - val_loss: 0.5155 - val_sparse_categorical_accuracy: 0.8495
Epoch 4/10
507/507 [==============================] - 12s 21ms/step - loss: 0.0861 - sparse_categorical_accuracy: 0.9678 - val_loss: 0.7043 - val_sparse_categorical_accuracy: 0.8184
Epoch 5/10
507/507 [==============================] - 12s 21ms/step - loss: 0.0615 - sparse_categorical_accuracy: 0.9795 - val_loss: 0.6418 - val_sparse_categorical_accuracy: 0.8501
Epoch 6/10
507/507 [==============================] - 11s 21ms/step - loss: 0.0470 - spars

### Evaluation on test dataset

In [ ]:
loss, acc = cnn2.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 6s 4ms/step - loss: 0.4420 - sparse_categorical_accuracy: 0.9052
Loss: 0.4420419931411743, Acc: 0.9052160382270813


# Experiments

## Using different number of scales in CMPooling (additional to the paper experiments)

We changed the scal_list variable to [1,3] and [1,3,5,7] to see the effect of using less and more scales on the accuracy.

### Using 2 scales: 1, 3

In [ ]:
SCALE_LIST = [1,3]
cnn2_2scales = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_2scales.fit(train_ds , epochs=10, validation_data=test_ds)

Epoch 1/10
507/507 [==============================] - 14s 23ms/step - loss: 1.0243 - sparse_categorical_accuracy: 0.5661 - val_loss: 0.7110 - val_sparse_categorical_accuracy: 0.7622
Epoch 2/10
507/507 [==============================] - 12s 22ms/step - loss: 0.1990 - sparse_categorical_accuracy: 0.9279 - val_loss: 0.6404 - val_sparse_categorical_accuracy: 0.8067
Epoch 3/10
507/507 [==============================] - 12s 22ms/step - loss: 0.1137 - sparse_categorical_accuracy: 0.9570 - val_loss: 0.7344 - val_sparse_categorical_accuracy: 0.8169
Epoch 4/10
507/507 [==============================] - 12s 22ms/step - loss: 0.0738 - sparse_categorical_accuracy: 0.9728 - val_loss: 0.8283 - val_sparse_categorical_accuracy: 0.8435
Epoch 5/10
507/507 [==============================] - 12s 22ms/step - loss: 0.0411 - sparse_categorical_accuracy: 0.9870 - val_loss: 0.8413 - val_sparse_categorical_accuracy: 0.8513
Epoch 6/10
507/507 [==============================] - 12s 22ms/step - loss: 0.0533 - spars

In [ ]:
loss, acc = cnn2_2scales.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 7s 5ms/step - loss: 0.9575 - sparse_categorical_accuracy: 0.8551
Loss: 0.9575446248054504, Acc: 0.8550925850868225


### Using 4 scales: 1, 3, 5, 7

In [ ]:
SCALE_LIST = [1,3,5,7]
cnn2_4scales = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_4scales.fit(train_ds , epochs=10, validation_data=test_ds)

Epoch 1/10
507/507 [==============================] - 11s 18ms/step - loss: 1.1150 - sparse_categorical_accuracy: 0.5220 - val_loss: 0.8657 - val_sparse_categorical_accuracy: 0.6835
Epoch 2/10
507/507 [==============================] - 10s 17ms/step - loss: 0.3297 - sparse_categorical_accuracy: 0.8811 - val_loss: 0.6372 - val_sparse_categorical_accuracy: 0.7795
Epoch 3/10
507/507 [==============================] - 10s 17ms/step - loss: 0.1634 - sparse_categorical_accuracy: 0.9405 - val_loss: 0.5707 - val_sparse_categorical_accuracy: 0.8196
Epoch 4/10
507/507 [==============================] - 10s 17ms/step - loss: 0.0964 - sparse_categorical_accuracy: 0.9671 - val_loss: 0.5090 - val_sparse_categorical_accuracy: 0.8432
Epoch 5/10
507/507 [==============================] - 10s 17ms/step - loss: 0.0720 - sparse_categorical_accuracy: 0.9741 - val_loss: 0.6125 - val_sparse_categorical_accuracy: 0.8418
Epoch 6/10
507/507 [==============================] - 10s 17ms/step - loss: 0.0439 - spars

In [ ]:
loss, acc = cnn2_4scales.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 6s 4ms/step - loss: 0.6039 - sparse_categorical_accuracy: 0.8728
Loss: 0.6038768291473389, Acc: 0.8727777600288391


We can see the accuracy when we use [1,3,5] as scale list is better than the other two.

## Pooling after Convolution

investigating the effect of placing cmpooling after conv layer on the accuracy

### Updated Monocular layer for Pooling after Convolution

In [ ]:
# Changing SCALE_LIST to the original one
SCALE_LIST = [1,3,5]
# Concat the feature maps in different scale and convolution once. (paper version)
class Monocular(tf.keras.layers.Layer):
    def __init__(self, filters, ksize, **kwargs):
        super(Monocular, self).__init__(**kwargs)
        self.filters = filters
        self.ksize = ksize
#         self.conv = tf.keras.layers.Conv2D(filters, ksize, input_shape=kwargs['input_shape'], activation='relu', padding='same')
    
    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(self.filters, self.ksize, input_shape=input_shape, activation='relu', padding='same')
    
    def call(self, fmaps, scale_list, pool_stride):
        conv_maps = self.conv(fmaps)
        pool_maps = cmpooling(conv_maps, scale_list, pool_stride)
        return tf.concat(pool_maps, axis=-1)

### Create and fitting the model

In [ ]:
cnn2_inverseP = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_inverseP.fit(train_ds , epochs=10, validation_data=test_ds)

Epoch 1/10
507/507 [==============================] - 14s 24ms/step - loss: 0.9562 - sparse_categorical_accuracy: 0.6003 - val_loss: 0.6173 - val_sparse_categorical_accuracy: 0.7788
Epoch 2/10
507/507 [==============================] - 12s 23ms/step - loss: 0.1988 - sparse_categorical_accuracy: 0.9290 - val_loss: 0.4227 - val_sparse_categorical_accuracy: 0.8585
Epoch 3/10
507/507 [==============================] - 13s 23ms/step - loss: 0.0936 - sparse_categorical_accuracy: 0.9689 - val_loss: 0.4928 - val_sparse_categorical_accuracy: 0.8618
Epoch 4/10
507/507 [==============================] - 13s 23ms/step - loss: 0.0495 - sparse_categorical_accuracy: 0.9828 - val_loss: 0.5512 - val_sparse_categorical_accuracy: 0.8547
Epoch 5/10
507/507 [==============================] - 13s 23ms/step - loss: 0.0383 - sparse_categorical_accuracy: 0.9875 - val_loss: 0.4510 - val_sparse_categorical_accuracy: 0.8826
Epoch 6/10
507/507 [==============================] - 13s 23ms/step - loss: 0.0435 - spars

### Evaluation on test dataset

In [ ]:
loss, acc = cnn2_inverseP.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 7s 4ms/step - loss: 0.5339 - sparse_categorical_accuracy: 0.8965
Loss: 0.5339114665985107, Acc: 0.8964506387710571


## Ablation study

### 1) Concentric multi-scale (CM) pooling contribution

Using Maxpooling instead of CMPooling

In [ ]:
class Monocular(tf.keras.layers.Layer):
    def __init__(self, filters, ksize, **kwargs):
        super(Monocular, self).__init__(**kwargs)
        self.filters = filters
        self.ksize = ksize
#         self.conv = tf.keras.layers.Conv2D(filters, ksize, input_shape=kwargs['input_shape'], activation='relu', padding='same')
    
    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(self.filters, self.ksize, input_shape=input_shape, activation='relu', padding='same')
    
    def call(self, fmaps, scale_list, pool_stride):
        pool_maps = tf.keras.layers.MaxPooling2D((2, 2), pool_stride, "SAME")(fmaps)
        return self.conv(pool_maps)

In [ ]:
cnn2_maxPool = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_maxPool.fit(train_ds , epochs=10, validation_data=test_ds)

Epoch 1/10
507/507 [==============================] - 14s 24ms/step - loss: 1.0394 - sparse_categorical_accuracy: 0.5524 - val_loss: 0.7981 - val_sparse_categorical_accuracy: 0.7282
Epoch 2/10
507/507 [==============================] - 13s 23ms/step - loss: 0.2383 - sparse_categorical_accuracy: 0.9103 - val_loss: 0.6835 - val_sparse_categorical_accuracy: 0.8091
Epoch 3/10
507/507 [==============================] - 13s 23ms/step - loss: 0.1272 - sparse_categorical_accuracy: 0.9553 - val_loss: 0.5733 - val_sparse_categorical_accuracy: 0.8375
Epoch 4/10
507/507 [==============================] - 13s 23ms/step - loss: 0.0861 - sparse_categorical_accuracy: 0.9702 - val_loss: 0.5843 - val_sparse_categorical_accuracy: 0.8514
Epoch 5/10
507/507 [==============================] - 13s 23ms/step - loss: 0.0497 - sparse_categorical_accuracy: 0.9842 - val_loss: 0.7617 - val_sparse_categorical_accuracy: 0.8436
Epoch 6/10
507/507 [==============================] - 13s 23ms/step - loss: 0.0451 - spars

In [ ]:
loss, acc = cnn2_maxPool.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 7s 5ms/step - loss: 0.8665 - sparse_categorical_accuracy: 0.8606
Loss: 0.866533637046814, Acc: 0.8606481552124023


### 2) Changing MaxPooling to AvgPooling in CMPooling layer (additional to the paper experiments)

In [ ]:
# Changing the Monocular to the original one
class Monocular(tf.keras.layers.Layer):
    def __init__(self, filters, ksize, **kwargs):
        super(Monocular, self).__init__(**kwargs)
        self.filters = filters
        self.ksize = ksize
#         self.conv = tf.keras.layers.Conv2D(filters, ksize, input_shape=kwargs['input_shape'], activation='relu', padding='same')
    
    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(self.filters, self.ksize, input_shape=input_shape, activation='relu', padding='same')
    
    def call(self, fmaps, scale_list, pool_stride):
        pool_maps = cmpooling(fmaps, scale_list, pool_stride)
        pool_maps = tf.concat(pool_maps, axis=-1)
        return self.conv(pool_maps)

In [ ]:
## Utility function
def cmpooling(fmaps, scale_list, pool_stride):
    # make sure the scale_list is in decending order
    if scale_list[0] - scale_list[1] < 0:
        scale_list = scale_list[::-1]
        
    # concentric multi-scale pooling
    offset = [0] + [-(scale_list[i+1] - scale_list[0])//2 for i in range(len(scale_list) - 1)]
    pool_maps = []
    for offset, scale in zip(offset, scale_list):
        slice_maps = tf.slice(fmaps, [0, offset, offset, 0], [-1, fmaps.shape[1]-offset*2, fmaps.shape[2]-offset*2, -1])
        pool_map = tf.nn.avg_pool2d(slice_maps, scale, pool_stride, "VALID")
        pool_maps.append(pool_map)
    
    # assert same shape for all pool_map
    for i in range(len(pool_maps)-1):
        assert pool_maps[i].shape[1:] == pool_maps[-1].shape[1:]
    return pool_maps

In [ ]:
cnn2_CM_avgPooling = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_CM_avgPooling.fit(train_ds , epochs=10, validation_data=test_ds)

Epoch 1/10
507/507 [==============================] - 12s 20ms/step - loss: 1.0428 - sparse_categorical_accuracy: 0.5758 - val_loss: 0.6871 - val_sparse_categorical_accuracy: 0.7395
Epoch 2/10
507/507 [==============================] - 11s 20ms/step - loss: 0.3307 - sparse_categorical_accuracy: 0.8771 - val_loss: 0.7388 - val_sparse_categorical_accuracy: 0.7624
Epoch 3/10
507/507 [==============================] - 11s 20ms/step - loss: 0.2052 - sparse_categorical_accuracy: 0.9265 - val_loss: 0.5369 - val_sparse_categorical_accuracy: 0.8194
Epoch 4/10
507/507 [==============================] - 11s 20ms/step - loss: 0.1282 - sparse_categorical_accuracy: 0.9573 - val_loss: 0.4935 - val_sparse_categorical_accuracy: 0.8413
Epoch 5/10
507/507 [==============================] - 11s 20ms/step - loss: 0.1045 - sparse_categorical_accuracy: 0.9632 - val_loss: 0.5676 - val_sparse_categorical_accuracy: 0.8315
Epoch 6/10
507/507 [==============================] - 11s 20ms/step - loss: 0.0756 - spars

In [ ]:
loss, acc = cnn2.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 6s 4ms/step - loss: 0.4420 - sparse_categorical_accuracy: 0.9052
Loss: 0.4420417547225952, Acc: 0.9052160382270813


### 3) One feedforward pathways

In [ ]:
# Changing the cmpooling to the original one
## Utility function
def cmpooling(fmaps, scale_list, pool_stride):
    # make sure the scale_list is in decending order
    if scale_list[0] - scale_list[1] < 0:
        scale_list = scale_list[::-1]
        
    # concentric multi-scale pooling
    offset = [0] + [-(scale_list[i+1] - scale_list[0])//2 for i in range(len(scale_list) - 1)]
    pool_maps = []
    for offset, scale in zip(offset, scale_list):
        slice_maps = tf.slice(fmaps, [0, offset, offset, 0], [-1, fmaps.shape[1]-offset*2, fmaps.shape[2]-offset*2, -1])
        pool_map = tf.nn.max_pool2d(slice_maps, scale, pool_stride, "VALID")
        pool_maps.append(pool_map)
    
    # assert same shape for all pool_map
    for i in range(len(pool_maps)-1):
        assert pool_maps[i].shape[1:] == pool_maps[-1].shape[1:]
    return pool_maps

In [ ]:
# Concat the feature maps in different scale and convolution once. (paper version)
class Monocular(tf.keras.layers.Layer):
    def __init__(self, filters, ksize, **kwargs):
        super(Monocular, self).__init__(**kwargs)
        self.filters = filters
        self.ksize = ksize
#         self.conv = tf.keras.layers.Conv2D(filters, ksize, input_shape=kwargs['input_shape'], activation='relu', padding='same')
    
    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(self.filters, self.ksize, input_shape=input_shape, activation='relu', padding='same')
    
    def call(self, fmaps, scale_list, pool_stride):
        conv_maps = self.conv(fmaps)
        pool_maps = tf.keras.layers.MaxPooling2D((2, 2), pool_stride, "SAME")(conv_maps)
        
        return pool_maps

In [ ]:
def CNN2(input_shape, num_classes, scale_list):
    left_eye = tf.keras.Input(input_shape, name='left_eye')
    right_eye = tf.keras.Input(input_shape, name='right_eye')
    
    x = tf.concat([left_eye, right_eye], axis=-1)

    x = Monocular(6, 5, input_shape=input_shape, name='mono1')(x, scale_list=scale_list, pool_stride=2)    
    x = Monocular(12, 5, name='mono2')(x, scale_list=scale_list, pool_stride=1)    
    x = Monocular(32, 3, name='mono3')(x, scale_list=scale_list, pool_stride=1)
    
    x = tf.keras.layers.Conv2D(256, 3, strides=1, activation='relu', name='conv1')(x)
    x = tf.keras.layers.Conv2D(256, 1, strides=1, activation='relu', name='conv2')(x)
    x = tf.keras.layers.Conv2D(64, 1, strides=1, activation='relu', name='conv3')(x)
    feature_vector = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predicted_output = tf.keras.layers.Dense(num_classes, activation='softmax', name='output')(feature_vector)
    
    return tf.keras.Model([left_eye, right_eye], predicted_output)

In [ ]:
cnn2_1fP = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_1fP.fit(train_ds , epochs=10, validation_data=test_ds)

Epoch 1/10
507/507 [==============================] - 12s 21ms/step - loss: 1.1706 - sparse_categorical_accuracy: 0.4964 - val_loss: 0.8464 - val_sparse_categorical_accuracy: 0.7144
Epoch 2/10
507/507 [==============================] - 12s 21ms/step - loss: 0.3333 - sparse_categorical_accuracy: 0.8750 - val_loss: 0.7090 - val_sparse_categorical_accuracy: 0.7817
Epoch 3/10
507/507 [==============================] - 12s 21ms/step - loss: 0.2220 - sparse_categorical_accuracy: 0.9184 - val_loss: 0.7315 - val_sparse_categorical_accuracy: 0.7889
Epoch 4/10
507/507 [==============================] - 12s 21ms/step - loss: 0.1519 - sparse_categorical_accuracy: 0.9478 - val_loss: 0.8273 - val_sparse_categorical_accuracy: 0.7989
Epoch 5/10
507/507 [==============================] - 11s 21ms/step - loss: 0.1051 - sparse_categorical_accuracy: 0.9655 - val_loss: 0.9048 - val_sparse_categorical_accuracy: 0.7932
Epoch 6/10
507/507 [==============================] - 12s 21ms/step - loss: 0.0927 - spars

In [ ]:
loss, acc = cnn2_1fP.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 7s 4ms/step - loss: 0.9860 - sparse_categorical_accuracy: 0.8262
Loss: 0.9860053062438965, Acc: 0.8262037038803101


### 4) Changing number of (CMPooling + conv) layers (additional to the paper experiments)

In [ ]:
# Changing the Monocular to the original one
# Concat the feature maps in different scale and convolution once. (paper version)
class Monocular(tf.keras.layers.Layer):
    def __init__(self, filters, ksize, **kwargs):
        super(Monocular, self).__init__(**kwargs)
        self.filters = filters
        self.ksize = ksize
#         self.conv = tf.keras.layers.Conv2D(filters, ksize, input_shape=kwargs['input_shape'], activation='relu', padding='same')
    
    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(self.filters, self.ksize, input_shape=input_shape, activation='relu', padding='same')
    
    def call(self, fmaps, scale_list, pool_stride):
        pool_maps = cmpooling(fmaps, scale_list, pool_stride)
        pool_maps = tf.concat(pool_maps, axis=-1)
        return self.conv(pool_maps)

2 (CMPooling + conv) layers 

In [ ]:
SCALE_LIST = [1,3,5]
def CNN2(input_shape, num_classes, scale_list):
    left_eye = tf.keras.Input(input_shape, name='left_eye')
    right_eye = tf.keras.Input(input_shape, name='right_eye')
    
    # parallax augmentation
    parallax = left_eye - right_eye 
    left = tf.concat([left_eye, -parallax], axis=-1)
    right = tf.concat([right_eye, parallax], axis=-1)
    # 
    left1 = Monocular(6, 5, input_shape=input_shape, name='mono1_left')(left, scale_list=scale_list, pool_stride=2)
    right1 = Monocular(6, 5, input_shape=input_shape, name='mono1_right')(right, scale_list=scale_list, pool_stride=2)
    
    left2 = Monocular(12, 5, name='mono2_left')(tf.concat([left1, right1], axis=-1), scale_list=scale_list, pool_stride=1)
    right2 = Monocular(12, 5, name='mono2_right')(tf.concat([right1, left1], axis=-1), scale_list=scale_list, pool_stride=1)
    
    x = tf.concat([left2, right2], axis=-1)
    x = tf.keras.layers.Conv2D(256, 3, strides=1, activation='relu', name='conv1')(x)
    x = tf.keras.layers.Conv2D(256, 1, strides=1, activation='relu', name='conv2')(x)
    x = tf.keras.layers.Conv2D(64, 1, strides=1, activation='relu', name='conv3')(x)
    feature_vector = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predicted_output = tf.keras.layers.Dense(num_classes, activation='softmax', name='output')(feature_vector)
    
    return tf.keras.Model([left_eye, right_eye], predicted_output)

In [ ]:
cnn2_2cm_conv_layers = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_2cm_conv_layers.fit(train_ds , epochs=10, validation_data=test_ds)

Epoch 1/10
507/507 [==============================] - 12s 19ms/step - loss: 0.9695 - sparse_categorical_accuracy: 0.6055 - val_loss: 0.6987 - val_sparse_categorical_accuracy: 0.7419
Epoch 2/10
507/507 [==============================] - 10s 19ms/step - loss: 0.2217 - sparse_categorical_accuracy: 0.9185 - val_loss: 0.7294 - val_sparse_categorical_accuracy: 0.7787
Epoch 3/10
507/507 [==============================] - 10s 19ms/step - loss: 0.1336 - sparse_categorical_accuracy: 0.9524 - val_loss: 0.5286 - val_sparse_categorical_accuracy: 0.8371
Epoch 4/10
507/507 [==============================] - 10s 19ms/step - loss: 0.0744 - sparse_categorical_accuracy: 0.9743 - val_loss: 0.6245 - val_sparse_categorical_accuracy: 0.8419
Epoch 5/10
507/507 [==============================] - 10s 19ms/step - loss: 0.0645 - sparse_categorical_accuracy: 0.9784 - val_loss: 0.5494 - val_sparse_categorical_accuracy: 0.8624
Epoch 6/10
507/507 [==============================] - 10s 19ms/step - loss: 0.0462 - spars

In [ ]:
loss, acc = cnn2_2cm_conv_layers.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 6s 4ms/step - loss: 0.6164 - sparse_categorical_accuracy: 0.8745
Loss: 0.6164230704307556, Acc: 0.8745370507240295


4 (CMPooling + conv) layers 

In [ ]:
SCALE_LIST = [1,3,5]
def CNN2(input_shape, num_classes, scale_list):
    left_eye = tf.keras.Input(input_shape, name='left_eye')
    right_eye = tf.keras.Input(input_shape, name='right_eye')
    
    # parallax augmentation
    parallax = left_eye - right_eye 
    left = tf.concat([left_eye, -parallax], axis=-1)
    right = tf.concat([right_eye, parallax], axis=-1)
    # 
    left1 = Monocular(6, 5, input_shape=input_shape, name='mono1_left')(left, scale_list=scale_list, pool_stride=2)
    right1 = Monocular(6, 5, input_shape=input_shape, name='mono1_right')(right, scale_list=scale_list, pool_stride=2)
    
    left2 = Monocular(12, 5, name='mono2_left')(tf.concat([left1, right1], axis=-1), scale_list=scale_list, pool_stride=1)
    right2 = Monocular(12, 5, name='mono2_right')(tf.concat([right1, left1], axis=-1), scale_list=scale_list, pool_stride=1)
    
    left3 = Monocular(32, 3, name='mono3_left')(tf.concat([left2, right2], axis=-1), scale_list=scale_list, pool_stride=1)
    right3 = Monocular(32, 3, name='mono3_right')(tf.concat([right2, left2], axis=-1), scale_list=scale_list, pool_stride=1)
    
    left4 = Monocular(64, 3, name='mono4_left')(tf.concat([left3, right3], axis=-1), scale_list=scale_list, pool_stride=1)
    right4 = Monocular(64, 3, name='mono4_right')(tf.concat([right3, left3], axis=-1), scale_list=scale_list, pool_stride=1)

    x = tf.concat([left4, right4], axis=-1)
    x = tf.keras.layers.Conv2D(256, 3, strides=1, activation='relu', name='conv1')(x)
    x = tf.keras.layers.Conv2D(256, 1, strides=1, activation='relu', name='conv2')(x)
    x = tf.keras.layers.Conv2D(64, 1, strides=1, activation='relu', name='conv3')(x)
    feature_vector = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predicted_output = tf.keras.layers.Dense(num_classes, activation='softmax', name='output')(feature_vector)
    
    return tf.keras.Model([left_eye, right_eye], predicted_output)

In [ ]:
cnn2_4cm_conv_layers = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_4cm_conv_layers.fit(train_ds , epochs=10, validation_data=test_ds)

Epoch 1/10
507/507 [==============================] - 13s 21ms/step - loss: 1.0019 - sparse_categorical_accuracy: 0.5833 - val_loss: 0.5754 - val_sparse_categorical_accuracy: 0.7765
Epoch 2/10
507/507 [==============================] - 11s 20ms/step - loss: 0.2219 - sparse_categorical_accuracy: 0.9154 - val_loss: 0.4977 - val_sparse_categorical_accuracy: 0.8405
Epoch 3/10
507/507 [==============================] - 11s 20ms/step - loss: 0.1058 - sparse_categorical_accuracy: 0.9622 - val_loss: 0.5933 - val_sparse_categorical_accuracy: 0.8346
Epoch 4/10
507/507 [==============================] - 11s 20ms/step - loss: 0.0551 - sparse_categorical_accuracy: 0.9819 - val_loss: 0.5200 - val_sparse_categorical_accuracy: 0.8628
Epoch 5/10
507/507 [==============================] - 11s 20ms/step - loss: 0.0664 - sparse_categorical_accuracy: 0.9749 - val_loss: 0.5164 - val_sparse_categorical_accuracy: 0.8757
Epoch 6/10
507/507 [==============================] - 11s 20ms/step - loss: 0.0308 - spars

In [ ]:
loss, acc = cnn2_4cm_conv_layers.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 6s 4ms/step - loss: 0.7442 - sparse_categorical_accuracy: 0.8605
Loss: 0.7442200779914856, Acc: 0.8604629635810852


### 5) Without augmentation

CNN2 without Augmentation

In [ ]:
SCALE_LIST = [1,3,5]
def CNN2(input_shape, num_classes, scale_list):
    left_eye = tf.keras.Input(input_shape, name='left_eye')
    right_eye = tf.keras.Input(input_shape, name='right_eye')
    
    # parallax augmentation
    left = left_eye
    right = right_eye
    # 
    left1 = Monocular(6, 5, input_shape=input_shape, name='mono1_left')(left, scale_list=scale_list, pool_stride=2)
    right1 = Monocular(6, 5, input_shape=input_shape, name='mono1_right')(right, scale_list=scale_list, pool_stride=2)
    
    left2 = Monocular(12, 5, name='mono2_left')(tf.concat([left1, right1], axis=-1), scale_list=scale_list, pool_stride=1)
    right2 = Monocular(12, 5, name='mono2_right')(tf.concat([right1, left1], axis=-1), scale_list=scale_list, pool_stride=1)
    
    left3 = Monocular(32, 3, name='mono3_left')(tf.concat([left2, right2], axis=-1), scale_list=scale_list, pool_stride=1)
    right3 = Monocular(32, 3, name='mono3_right')(tf.concat([right2, left2], axis=-1), scale_list=scale_list, pool_stride=1)
    
    x = tf.concat([left3, right3], axis=-1)
    x = tf.keras.layers.Conv2D(256, 3, strides=1, activation='relu', name='conv1')(x)
    x = tf.keras.layers.Conv2D(256, 1, strides=1, activation='relu', name='conv2')(x)
    x = tf.keras.layers.Conv2D(64, 1, strides=1, activation='relu', name='conv3')(x)
    feature_vector = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predicted_output = tf.keras.layers.Dense(num_classes, activation='softmax', name='output')(feature_vector)
    
    return tf.keras.Model([left_eye, right_eye], predicted_output)

In [ ]:
cnn2_WoAug = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_WoAug.fit(train_ds , epochs=10, validation_data=test_ds)

Epoch 1/10
507/507 [==============================] - 13s 21ms/step - loss: 1.0976 - sparse_categorical_accuracy: 0.5491 - val_loss: 0.8798 - val_sparse_categorical_accuracy: 0.7065
Epoch 2/10
507/507 [==============================] - 11s 21ms/step - loss: 0.2999 - sparse_categorical_accuracy: 0.8925 - val_loss: 0.7536 - val_sparse_categorical_accuracy: 0.7510
Epoch 3/10
507/507 [==============================] - 12s 21ms/step - loss: 0.1528 - sparse_categorical_accuracy: 0.9476 - val_loss: 0.6686 - val_sparse_categorical_accuracy: 0.8040
Epoch 4/10
507/507 [==============================] - 12s 21ms/step - loss: 0.0866 - sparse_categorical_accuracy: 0.9708 - val_loss: 0.6138 - val_sparse_categorical_accuracy: 0.8145
Epoch 5/10
507/507 [==============================] - 11s 21ms/step - loss: 0.0601 - sparse_categorical_accuracy: 0.9795 - val_loss: 0.6524 - val_sparse_categorical_accuracy: 0.8330
Epoch 6/10
507/507 [==============================] - 11s 21ms/step - loss: 0.0482 - spars

In [ ]:
loss, acc = cnn2_WoAug.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 7s 5ms/step - loss: 0.9176 - sparse_categorical_accuracy: 0.8323
Loss: 0.9175538420677185, Acc: 0.8323456645011902


## Fusion of the Two Feedforward Pathways

In [ ]:
SCALE_LIST = [1,3,5]
def CNN2(input_shape, num_classes, scale_list):
    left_eye = tf.keras.Input(input_shape, name='left_eye')
    right_eye = tf.keras.Input(input_shape, name='right_eye')
    
    # parallax augmentation
    parallax = left_eye - right_eye 
    left = tf.concat([left_eye, -parallax], axis=-1)
    right = tf.concat([right_eye, parallax], axis=-1)
    # 
    
    left1 = Monocular(6, 5, input_shape=input_shape, name='mono1_left')(left, scale_list=scale_list, pool_stride=2)
    right1 = Monocular(6, 5, input_shape=input_shape, name='mono1_right')(right, scale_list=scale_list, pool_stride=2)

    left2 = Monocular(12, 5, name='mono2_left')(left1, scale_list=scale_list, pool_stride=1)
    right2 = Monocular(12, 5, name='mono2_right')(right1, scale_list=scale_list, pool_stride=1)

    left3 = Monocular(32, 3, name='mono3_left')(left2, scale_list=scale_list, pool_stride=1)
    right3 = Monocular(32, 3, name='mono3_right')(right2, scale_list=scale_list, pool_stride=1)
    
    x = tf.concat([left3, right3], axis=-1)
    x = tf.keras.layers.Conv2D(256, 3, strides=1, activation='relu', name='conv1')(x)
    x = tf.keras.layers.Conv2D(256, 1, strides=1, activation='relu', name='conv2')(x)
    x = tf.keras.layers.Conv2D(64, 1, strides=1, activation='relu', name='conv3')(x)
    feature_vector = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predicted_output = tf.keras.layers.Dense(num_classes, activation='softmax', name='output')(feature_vector)
    
    return tf.keras.Model([left_eye, right_eye], predicted_output)

In [ ]:
cnn2_fu = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_fu.fit(train_ds , epochs=10, validation_data=test_ds)

Epoch 1/10
507/507 [==============================] - 12s 21ms/step - loss: 1.0221 - sparse_categorical_accuracy: 0.5821 - val_loss: 0.8269 - val_sparse_categorical_accuracy: 0.7288
Epoch 2/10
507/507 [==============================] - 11s 20ms/step - loss: 0.2676 - sparse_categorical_accuracy: 0.9014 - val_loss: 0.8192 - val_sparse_categorical_accuracy: 0.7505
Epoch 3/10
507/507 [==============================] - 11s 20ms/step - loss: 0.1627 - sparse_categorical_accuracy: 0.9414 - val_loss: 0.7982 - val_sparse_categorical_accuracy: 0.8021
Epoch 4/10
507/507 [==============================] - 11s 19ms/step - loss: 0.1077 - sparse_categorical_accuracy: 0.9623 - val_loss: 0.6802 - val_sparse_categorical_accuracy: 0.8273
Epoch 5/10
507/507 [==============================] - 11s 20ms/step - loss: 0.0668 - sparse_categorical_accuracy: 0.9792 - val_loss: 0.5645 - val_sparse_categorical_accuracy: 0.8507
Epoch 6/10
507/507 [==============================] - 11s 20ms/step - loss: 0.0492 - spars

In [ ]:
loss, acc = cnn2_fu.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 7s 5ms/step - loss: 0.8174 - sparse_categorical_accuracy: 0.8436
Loss: 0.8174134492874146, Acc: 0.8435802459716797


# Vanilla CNN

In [ ]:
def CNN(input_shape, num_classes):
    
    left_eye= tf.keras.Input(shape=input_shape, name = 'left_eye')
    right_eye= tf.keras.Input(shape=input_shape, name = 'right_eye')
    
    parallax = left_eye - right_eye 
    x = tf.concat([left_eye, -parallax], axis=-1)
    
    x = tf.keras.layers.Conv2D(6, 5,  activation='relu', padding= 'same', name = 'mono1_left')(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Conv2D(12, 5, activation='relu', padding= 'same', name='mono2_left')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(1,1))(x)
    x = tf.keras.layers.Conv2D(32, 3, activation='relu', padding= 'same', name='mono3_left')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(1,1))(x)
    x = tf.keras.layers.Conv2D(256, 3, strides=1, activation='relu', name='conv1')(x)
    x = tf.keras.layers.Conv2D(256, 1, strides=1, activation='relu', name='conv2')(x)
    x = tf.keras.layers.Conv2D(64, 1, strides=1, activation='relu', name='conv3')(x)
    feature_vector = tf.keras.layers.Flatten()(x)
    predicted_output = tf.keras.layers.Dense(num_classes, activation="softmax", name='output')(feature_vector)
    return tf.keras.Model([left_eye, right_eye], predicted_output)

In [ ]:
def create_model(model, input_shape, num_classes):
    m = model(input_shape, num_classes)
    # learning rate schedule
    initial_learning_rate = 0.0001
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=100000,
                                                              decay_rate=0.96, staircase=True)
    
    # compile the model
    m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),  # Optimizer
                  # Loss function to minimize
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  # List of metrics to monitor
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
                 )
    return m

In [ ]:
start = time.time()
cnn = create_model(CNN, input_shape=(IMG_SIZE, IMG_SIZE, 1), num_classes=5)
cnn.fit(train_ds , epochs=10, validation_data=test_ds)
end = time.time()
print(f'Running time: {end-start}')

Epoch 1/10
507/507 [==============================] - 14s 24ms/step - loss: 1.2436 - sparse_categorical_accuracy: 0.4623 - val_loss: 0.9235 - val_sparse_categorical_accuracy: 0.6656
Epoch 2/10
507/507 [==============================] - 13s 23ms/step - loss: 0.3451 - sparse_categorical_accuracy: 0.8781 - val_loss: 0.6198 - val_sparse_categorical_accuracy: 0.7811
Epoch 3/10
507/507 [==============================] - 13s 23ms/step - loss: 0.1484 - sparse_categorical_accuracy: 0.9507 - val_loss: 0.5113 - val_sparse_categorical_accuracy: 0.8454
Epoch 4/10
507/507 [==============================] - 13s 23ms/step - loss: 0.0871 - sparse_categorical_accuracy: 0.9720 - val_loss: 0.6564 - val_sparse_categorical_accuracy: 0.8391
Epoch 5/10
507/507 [==============================] - 12s 22ms/step - loss: 0.0549 - sparse_categorical_accuracy: 0.9833 - val_loss: 0.5858 - val_sparse_categorical_accuracy: 0.8488
Epoch 6/10
507/507 [==============================] - 12s 22ms/step - loss: 0.0393 - spars

In [ ]:
loss, acc = cnn.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 8s 5ms/step - loss: 1.3307 - sparse_categorical_accuracy: 0.7961
Loss: 1.3306844234466553, Acc: 0.7960802316665649


## VanillaCNN with CMPooling

In [ ]:
# Concat the feature maps in different scale and convolution once. (paper version)
class Monocular(tf.keras.layers.Layer):
    def __init__(self, filters, ksize, **kwargs):
        super(Monocular, self).__init__(**kwargs)
        self.filters = filters
        self.ksize = ksize
#         self.conv = tf.keras.layers.Conv2D(filters, ksize, input_shape=kwargs['input_shape'], activation='relu', padding='same')
    
    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(self.filters, self.ksize, input_shape=input_shape, activation='relu', padding='same')
    
    def call(self, fmaps, scale_list, pool_stride):
        conv_maps = self.conv(fmaps)
        pool_maps = cmpooling(conv_maps, scale_list, pool_stride)
        return tf.concat(pool_maps, axis=-1)

In [ ]:
SCALE_LIST = [1,3,5]
def CNN(input_shape, num_classes, scale_list):
    
    left_eye= tf.keras.Input(shape=input_shape, name = 'left_eye')
    right_eye= tf.keras.Input(shape=input_shape, name = 'right_eye')
    
    parallax = left_eye - right_eye 
    left = tf.concat([left_eye, -parallax], axis=-1)

    y = Monocular(6, 5, input_shape=input_shape, name='mono1_left')(left, scale_list=scale_list, pool_stride=2)
    y = Monocular(12, 5, input_shape=input_shape, name='mono2_left')(y, scale_list=scale_list, pool_stride=1)
    y = Monocular(32, 3, input_shape=input_shape, name='mono3_left')(y, scale_list=scale_list, pool_stride=1)
    
    y = tf.keras.layers.Conv2D(256, 3, strides=1, activation='relu', name='conv1')(y)
    y = tf.keras.layers.Conv2D(256, 1, strides=1, activation='relu', name='conv2')(y)
    y = tf.keras.layers.Conv2D(64, 1, strides=1, activation='relu', name='conv3')(y)
    feature_vector = tf.keras.layers.Flatten()(y)
    predicted_output = tf.keras.layers.Dense(num_classes, activation="softmax", name='output')(feature_vector)
    return tf.keras.Model([left_eye, right_eye], predicted_output)

In [ ]:
def create_model(model, input_shape, num_classes, scale_list):
    m = model(input_shape, num_classes, scale_list)
    # learning rate schedule
    initial_learning_rate = 0.0001
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=100000,
                                                              decay_rate=0.96, staircase=True)
    
    # compile the model
    m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),  # Optimizer
                  # Loss function to minimize
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  # List of metrics to monitor
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
                 )
    return m

In [ ]:
cnn_cmp = create_model(CNN, input_shape=(IMG_SIZE, IMG_SIZE, 1), num_classes=5, scale_list=SCALE_LIST)
cnn_cmp.fit(train_ds , epochs=10, validation_data=test_ds, shuffle=False)

Epoch 1/10
507/507 [==============================] - 12s 19ms/step - loss: 1.0960 - sparse_categorical_accuracy: 0.5486 - val_loss: 0.7567 - val_sparse_categorical_accuracy: 0.7288
Epoch 2/10
507/507 [==============================] - 11s 19ms/step - loss: 0.3166 - sparse_categorical_accuracy: 0.8845 - val_loss: 0.6075 - val_sparse_categorical_accuracy: 0.7898
Epoch 3/10
507/507 [==============================] - 10s 19ms/step - loss: 0.2001 - sparse_categorical_accuracy: 0.9257 - val_loss: 0.5295 - val_sparse_categorical_accuracy: 0.8120
Epoch 4/10
507/507 [==============================] - 11s 19ms/step - loss: 0.1661 - sparse_categorical_accuracy: 0.9429 - val_loss: 0.5143 - val_sparse_categorical_accuracy: 0.8313
Epoch 5/10
507/507 [==============================] - 11s 19ms/step - loss: 0.0933 - sparse_categorical_accuracy: 0.9695 - val_loss: 0.5681 - val_sparse_categorical_accuracy: 0.8394
Epoch 6/10
507/507 [==============================] - 10s 19ms/step - loss: 0.0926 - spars

In [ ]:
loss, acc = cnn_cmp.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 7s 5ms/step - loss: 0.4849 - sparse_categorical_accuracy: 0.8810
Loss: 0.48494547605514526, Acc: 0.881049394607544
